In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

import pandas as pd
import numpy as np
from func_linear import *

import warnings
warnings.filterwarnings('ignore')

In [15]:
x = pd.read_csv('train.csv')
y = x[x.columns[-1]]
df_test = pd.read_csv('test.csv')

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
l1 = {1: '20-30', 2: '30-40', 3: '40-50', 4: '50-60', 5: '60-70', 6: '70-80'}
l3 = {0: '0', 1: '1 - 10', 2: '11 - 23', 3: '24 - 36', 4: '37 - 49', 5: '50 - 62', 6: '63 - 75', 7: '76 - 88', 8: '89 - 99', 9: '100'}
l4 = {0: '0', 1: '1-49', 2: '50-99', 3: '100-199', 4: '200-499', 5: '500-999', 6: '1000-4999', 7: '5000-9999', 8: '10000-19999', 9: '20000'}

In [4]:
pipe = Pipeline([
    ('dummy1', create_dummy('Customer_Subtype')),
    ('dummy2', create_dummy('Customer_main_type')),
    ('custom1', custom('Avg_age', l1)),
    ('custom2', custom('Roman_catholic', l3)),
    ('custom3', custom('Contribution_private_third_party_insurance', l4))
])

In [5]:
pipe.fit(x)

Pipeline(steps=[('dummy1', create_dummy(var='Customer_Subtype')),
                ('dummy2', create_dummy(var='Customer_main_type')),
                ('custom1',
                 custom(pairs={1: '20-30', 2: '30-40', 3: '40-50', 4: '50-60',
                               5: '60-70', 6: '70-80'},
                        var='Avg_age')),
                ('custom2',
                 custom(pairs={0: '0', 1: '1 - 10', 2: '11 - 23', 3: '24 - 36',
                               4: '37 - 49', 5: '50 - 62', 6: '63 - 75',
                               7: '76 - 88', 8: '89 - 99', 9: '100'},
                        var='Roman_catholic')),
                ('custom3',
                 custom(pairs={0: '0', 1: '1-49', 2: '50-99', 3: '100-199',
                               4: '200-499', 5: '500-999', 6: '1000-4999',
                               7: '5000-9999', 8: '10000-19999', 9: '20000'},
                        var='Contribution_private_third_party_insurance'))])

In [19]:
x.columns


Index(['Customer_Subtype', 'Number_of_houses_1_–_10',
       'Avg_size_household_1_–_6', 'Avg_age', 'Customer_main_type',
       'Roman_catholic', 'Protestant_...', 'Other_religion', 'No_religion',
       'Married', 'Living_together', 'Other_relation', 'Singles',
       'Household_without_children', 'Household_with_children',
       'High_level_education', 'Medium_level_education',
       'Lower_level_education', 'High_status', 'Entrepreneur', 'Farmer',
       'Middle_management', 'Skilled_labourers', 'Unskilled_labourers',
       'Social_class_A', 'Social_class_B1', 'Social_class_B2',
       'Social_class_C', 'Social_class_D', 'Rented_house', 'Home_owners',
       '1_car', '2_cars', 'No_car', 'National_Health_Service',
       'Private_health_insurance', 'Income_<_30.000', 'Income_30-45.000',
       'Income_45-75.000', 'Income_75-122.000', 'Income_>123.000',
       'Average_income', 'Purchasing_power_class',
       'Contribution_private_third_party_insurance',
       'Contribution_thir

In [11]:
y_train = x['V86'].iloc[1160:]
y_test = x['V86'].iloc[0:1160]
del x['V86']
X_train = x.iloc[1160:]
X_test = x.iloc[0:1160]

KeyError: 'V86'

In [ ]:
x_train = pipe.transform(X_train)
x_test = pipe.transform(X_test)

In [ ]:
# params={'class_weight':[None],
#         'penalty':['l1','l2'],
#         'C':[.0001,.0005,.001,.005,.01,.05,.1,1,2,5]}

# model=LogisticRegression(fit_intercept=True)

# grid_search=GridSearchCV(model,
#                          param_grid=params,
#                          cv=10,
#                          scoring="roc_auc",
#                          n_jobs=-1,
#                          verbose=20)

clf = RandomForestClassifier()
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None],
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

n_iter_search = 10
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   scoring='roc_auc',
                                   cv=10,
                                  n_jobs=-1,verbose=20)

In [ ]:
random_search.fit(x_train, y_train)

In [ ]:
# grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
report(random_search.cv_results_, 3)

In [ ]:
logr = RandomForestClassifier(**{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 15, 
                                 'max_features': 20, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True})

In [ ]:
logr.fit(x_train, y_train)

In [ ]:
train_score=logr.predict_proba(x_test[imp_feat])[:,1]
real=y_test

cutoffs=np.linspace(0.01,0.99,99)

In [ ]:
f_beta_score={}

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP

    precision = (TP/TP+FP)*(TP/TP+FN)
    recall = (TP/TP+FP)+(TP/TP+FN)
    
      
    f_beta=((1+4)*precision)/recall
       
    f_beta_score[cutoff] = f_beta

In [ ]:
for i in f_beta_score.keys():
    if f_beta_score[i] == max(f_beta_score.values()):
        print(i, f_beta_score[i])

In [ ]:
test = pipe.transform(df_test)

In [ ]:
predicted = (logr.predict_proba(test[imp_feat])[:,1]>0.15).astype(int)

In [ ]:
pd.DataFrame(predicted).to_csv('submission.csv', index=False)

In [ ]:
predicted.sum()

In [ ]:
(170/test.shape[0])*100

In [ ]:
f_beta_score

In [ ]:
15*12

In [ ]:
1.8+1.8

In [ ]:
5.4-3.6

In [ ]:
(x['V86'].sum()/len(x))*100

In [ ]:
len(x)

In [ ]:
feat_imp_df=pd.DataFrame({'features':x_train.columns,
                          'importance':logr.feature_importances_})

feat_imp_df=feat_imp_df.sort_values('importance',ascending=False)
feat_imp_df['normalised_imp']=feat_imp_df['importance']/np.sum(feat_imp_df['importance'])
feat_imp_df['cum_imp']=np.cumsum(feat_imp_df['normalised_imp'])

imp_feat = list(feat_imp_df.head(50)['features'])

imp_feat_df = pd.concat([x_train, x_test])[imp_feat]

In [ ]:
logr.fit(x_train[imp_feat], y_train)